In [2]:
import ipywidgets as widgets
from ast import literal_eval
import sys
import configparser
from selectfile import FileBrowser

The following cell can be modified to determine usage of an imported ini, as well as change the names and paths of import and export files.

In [5]:
use_import = True
ini_import = "default.ini"
ini_export = "example.ini"

if use_import:
    defaultconf = configparser.ConfigParser()
    defaultconf.read(ini_import)

In [6]:
#Stores method names, exactly as expected in .ini file
methods = ["1NN", "KKNN", "RF", "HYPPO", "UNMODEL"]
#Stores the names of variables that need to be expressed as file paths
paths = ["START", "CODE", "DATA", "SHAPES", "DOMAINS", "OUTPUT", "SM_FILE", "COV_FILE", "EVAL_FILE"]
items = []
tempdict = {}

#Sets up the file selector widgets as a dictionary, with or without default input. This is important, as the FileSelector is 
#class with multiple widgets at work - we need an original instance to properly retrieve the input. Using the traditional
#ipython widget value returns incorrectly.
path_widgets = {}
if use_import:
    for j in paths:
        path_widgets[j] = FileBrowser(start_path=defaultconf["DEFAULT"][j])
else:
    for j in paths:
        path_widgets[j] = FileBrowser()
#Makes a VBox out of HBoxes of widgets for method arguments, for addition to the input GUI.
for step, i in enumerate(methods):
    boxpop = (widgets.Label(i), widgets.Checkbox(), widgets.Text())
    items.append(widgets.HBox(boxpop))
modict = widgets.VBox(items)



# This has been replaced with a dictionary-fed version below to allow for easy updating.
#inputs = widgets.Accordion(children=[widgets.Text(), widgets.Text(), widgets.Text(), widgets.Text(), widgets.Text(), 
#                                       widgets.Text(), widgets.IntText(), widgets.Text(), widgets.Text(), widgets.Text(), widgets.Text(), 
#                                       widgets.Checkbox(), widgets.Checkbox(), widgets.FloatText(), widgets.IntText(), widgets.Checkbox(),
#                                       widgets.IntSlider(), widgets.Text(), widgets.Text(), modict
#                                       ])

#All the variable names, for easier config parsing.
titles = ["START", "CODE", "DATA", "SHAPES", "DOMAINS", "OUTPUT", "YEAR", "MONTHS", "SM_FILE", "COV_FILE", "EVAL_FILE", 
         "MAKE_T_E", "USE_PCA", "VALIDATE", "RAND_SEED", "USE_VIS", "MIN_TEST_POINTS", "BUFFER", "SUPER", "MODICT"]

#A list of variables and their associated widgets, in a dictionary. 
child_type = {"START":path_widgets["START"].widget(), "CODE":path_widgets["CODE"].widget(), "DATA":path_widgets["DATA"].widget(), 
                 "SHAPES":path_widgets["SHAPES"].widget(), "DOMAINS":path_widgets["DOMAINS"].widget(), "OUTPUT":path_widgets["OUTPUT"].widget(), 
                 "YEAR":widgets.IntText(), "MONTHS":widgets.Text(), "SM_FILE":path_widgets["SM_FILE"].widget(), 
                 "COV_FILE":path_widgets["COV_FILE"].widget(), "EVAL_FILE":path_widgets["EVAL_FILE"].widget(), "MAKE_T_E":widgets.Checkbox(), 
                 "USE_PCA":widgets.Checkbox(), "VALIDATE":widgets.FloatText(), "RAND_SEED":widgets.IntText(), 
                 "USE_VIS":widgets.Checkbox(), "MIN_TEST_POINTS": widgets.IntSlider(), "BUFFER":widgets.Text(), 
                 "SUPER":widgets.Checkbox(), "MODICT":modict
               }

inputs = widgets.Accordion(children=[child_type[title] for title in titles])

# The following dictionary has a str-type description for each parameter
descriptions = {"START":"(string or path) The main working directory, containing relevant code and data subdirectories.", 
                "CODE":"(string or path) The name of the START-subdirectory with code.", 
                "DATA":"(string or path) The name of the START-subdirectory for data.",
                "SHAPES":"(string or path) The name of the DATA-subdirectory for region shapefiles.", 
                "DOMAINS": "(array of strings) A list of all domains.", 
                "OUTPUT": "", 
                "YEAR": "", 
                "MONTHS": "", 
                "SM_FILE": "", 
                "COV_FILE": "", 
                "EVAL_FILE": "", 
                "MAKE_T_E": "", 
                "USE_PCA": "", 
                "VALIDATE": "", 
                "RAND_SEED": "", 
                "USE_VIS": "", 
                "MIN_TEST_POINTS": "", 
                "BUFFER": "", 
                "SUPER": "", 
                "MODICT": ""
               }
#Assigns names to our Accordion entries.
for index, j in enumerate(titles):
    inputs.set_title(index, j)
    
#The following code fills the widgets with the input from the imported .ini, if one was imported.
if use_import:
    for index, k in enumerate(titles[:-1]):
        if k in ["USE_PCA", "MAKE_T_E", "USE_VIS", "SUPER"]:
            inputs.children[index].value = bool(int(defaultconf["DEFAULT"][k]))
        else:
            inputs.children[index].value = defaultconf["DEFAULT"][k]

    tempdict = literal_eval(defaultconf["DEFAULT"][titles[-1]])
    if tempdict:
        for index, s in enumerate(methods):
            if s in tempdict:
                inputs.children[-1].children[index].children[2].value = str(tempdict[s])
                inputs.children[-1].children[index].children[1].value = True
                                               

display(inputs)

Accordion(children=(VBox(children=(HTML(value='<h2>..</h2>'), Button(description='..', style=ButtonStyle()), B…

In [4]:
#Building the output file.

config = configparser.ConfigParser()
config["DEFAULT"] = {}
for index, j in enumerate(titles[:-1]):
    #Boolean variables
    if j in ["MAKE_T_E", "USE_PCA", "RAND_SEED", "USE_VIS"]:
        config["DEFAULT"][j] = str(int(inputs.children[index].value))
    #File paths from FileSelector
    elif j in paths:
        config["DEFAULT"][j] = path_widgets[j].path
    #Everything else.
    else:
        config["DEFAULT"][j] = str(inputs.children[index].value)
        
#Filling the ML methods output.
modict = "{\n"
for learners in inputs.children[-1].children:
    if learners.children[1].value:
        modict += f"\"{learners.children[0].value}\":{learners.children[2].value}\n"
modict += "}"
config["DEFAULT"][titles[-1]] = modict

with open(ini_export, 'w') as file:
    config.write(file)

In [5]:
print(str(path_widgets["START"].path))

..


In [20]:
print(inputs.children[0].children[0].children[1].value)

AttributeError: 'VBox' object has no attribute 'value'